In [144]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
import quandl

sns.set()

In [145]:
#quandl.ApiConfig.api_key = "ByAzesCp4TpPvjqYi4ay" #Pour faire plus de 50 requetes par jours
#start = datetime(2016,1,1)
#end = datetime(2017,1,1)
#s = "CBS"

In [146]:
logo_all=['AAPL','ABT','ACN','ATVI','ADBE','AMD','GOOGL','GOOG','AMZN','AAL','BLK','CBS','MSFT','FB']
logo=logo_all[1:]

logo_pred=['AAPL_Pred']
for l in logo:
    logo_pred.append(l+'_Pred')
    

    
logo_pred

['AAPL_Pred',
 'ABT_Pred',
 'ACN_Pred',
 'ATVI_Pred',
 'ADBE_Pred',
 'AMD_Pred',
 'GOOGL_Pred',
 'GOOG_Pred',
 'AMZN_Pred',
 'AAL_Pred',
 'BLK_Pred',
 'CBS_Pred',
 'MSFT_Pred',
 'FB_Pred']

In [147]:
df_companies=pd.DataFrame()
df_companies['logo']=logo_all
df_companies['logo_pred']=logo_pred
df_companies.to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Liste_entreprise.csv")




In [148]:
df_all=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_total.csv")
df=df_all[:-3] #Les data - les 3 à prédire !
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()


In [149]:
df.tail(10)

,Date,AAPL,ABT,ACN,ATVI,ADBE,AMD,GOOGL,GOOG,AMZN,AAL,BLK,CBS,MSFT,FB
244,2015-12-21,107.33,44.04,102.99,38.74,91.62,2.53,760.80,747.77,664.51,42.400,325.55,45.79,54.83,104.77
245,2015-12-22,107.23,44.48,103.76,39.21,93.82,2.77,767.13,750.00,663.15,42.895,330.51,46.61,55.35,105.51
246,2015-12-23,108.61,45.10,104.42,38.84,94.70,2.83,768.51,750.31,663.70,43.290,339.64,47.23,55.82,104.63
247,2015-12-24,108.03,45.10,104.23,38.92,94.30,2.92,765.84,748.40,662.79,43.810,337.40,47.44,55.67,105.02
248,2015-12-28,106.82,45.03,104.08,39.03,94.20,3.00,782.24,762.51,675.20,43.230,342.45,46.83,55.95,105.93
249,2015-12-29,108.74,45.82,105.32,39.58,95.33,3.00,793.96,776.60,693.97,43.500,343.60,47.29,56.55,107.26
250,2015-12-30,107.32,45.27,105.86,39.43,95.28,2.98,790.30,771.00,689.07,42.800,342.05,46.76,56.31,106.22
251,2015-12-31,105.26,44.91,104.50,38.71,93.94,2.87,778.01,758.88,675.89,42.350,340.52,47.13,55.48,104.66
252,2016-01-04,105.35,42.93,101.83,37.62,91.97,2.77,759.44,741.84,636.99,40.910,333.10,46.65,54.80,102.22
253,2016-01-05,102.71,42.92,102.36,37.14,92.34,2.75,761.53,742.58,633.79,40.520,333.96,46.12,55.05,102.73


# Essayer avec une dataframe avec Data . Close Ent1, Close Ent2, Close Ent3, Close Ent4,.....)

In [150]:
minmax = MinMaxScaler().fit(df.iloc[:, 1:].astype('float32'))
df_log = minmax.transform(df.iloc[:, 1:].astype('float32'))
df_log = pd.DataFrame(df_log)



C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Idée :Faire le [:3] après le MinMaxScaller pour avoir les 3 derniers scaller aussi ? 

In [151]:
df_log.tail()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
249,0.199076,0.556837,0.828405,1.000000,0.991005,0.816568,1.000000,1.000000,1.000000,0.328587,0.576892,0.349271,1.000000,0.949943
250,0.152195,0.511532,0.849416,0.992867,0.989050,0.804734,0.987673,0.980285,0.987961,0.290252,0.559037,0.327796,0.985240,0.920194
251,0.084186,0.481878,0.796498,0.958630,0.936645,0.739645,0.946278,0.937617,0.955580,0.265608,0.541412,0.342788,0.934194,0.875572
252,0.087157,0.318781,0.692607,0.906800,0.859601,0.680473,0.883732,0.877627,0.860007,0.186747,0.455938,0.323339,0.892374,0.805778
253,0.000000,0.317957,0.713230,0.883975,0.874071,0.668639,0.890771,0.880232,0.852145,0.165389,0.465845,0.301864,0.907749,0.820366


In [152]:
class Model:
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)

        rnn_cells = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell(size_layer) for _ in range(num_layers)],
            state_is_tuple = False,
        )
        self.X = tf.placeholder(tf.float32, (None, None, size))
        self.Y = tf.placeholder(tf.float32, (None, output_size))
        drop = tf.contrib.rnn.DropoutWrapper(
            rnn_cells, output_keep_prob = forget_bias
        )
        self.hidden_layer = tf.placeholder(
            tf.float32, (None, num_layers * 2 * size_layer)
        )
        self.outputs, self.last_state = tf.nn.dynamic_rnn(
            drop, self.X, initial_state = self.hidden_layer, dtype = tf.float32
        )
        self.logits = tf.layers.dense(self.outputs[-1], output_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )

In [153]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 500
dropout_rate = 0.7
future_day = 5

In [154]:
tf.reset_default_graph()
modelnn = Model(
    0.01, num_layers, df_log.shape[1], size_layer, df_log.shape[1], dropout_rate
)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [155]:
for i in range(epoch):
    init_value = np.zeros((1, num_layers * 2 * size_layer))
    total_loss = 0
    for k in range(0, df_log.shape[0] - 1, timestamp):
        index = min(k + timestamp, df_log.shape[0] -1)
        batch_x = np.expand_dims(
            df_log.iloc[k : index, :].values, axis = 0
        )
        batch_y = df_log.iloc[k + 1 : index + 1, :].values
        last_state, _, loss = sess.run(
            [modelnn.last_state, modelnn.optimizer, modelnn.cost],
            feed_dict = {
                modelnn.X: batch_x,
                modelnn.Y: batch_y,
                modelnn.hidden_layer: init_value,
            },
        )
        init_value = last_state
        total_loss += loss
    total_loss /= df_log.shape[0] // timestamp
    if (i + 1) % 100 == 0:
        print('epoch:', i + 1, 'avg loss:', total_loss)

epoch: 100 avg loss: 0.014415911342948675
epoch: 200 avg loss: 0.011466263150796295
epoch: 300 avg loss: 0.010495326207019389
epoch: 400 avg loss: 0.00872368274256587
epoch: 500 avg loss: 0.009625736605376006


In [156]:
len(df_log)

254

In [157]:
output_predict = np.zeros((df_log.shape[0] + future_day, df_log.shape[1]))
output_predict[0] = df_log.iloc[0]
upper_b = (df_log.shape[0] // timestamp) * timestamp
init_value = np.zeros((1, num_layers * 2 * size_layer))
for k in range(0, (df_log.shape[0] // timestamp) * timestamp, timestamp):
    out_logits, last_state = sess.run(
        [modelnn.logits, modelnn.last_state],
        feed_dict = {
            modelnn.X: np.expand_dims(
                df_log.iloc[k : k + timestamp], axis = 0
            ),
            modelnn.hidden_layer: init_value,
        },
    )
    init_value = last_state
    output_predict[k + 1 : k + timestamp + 1] = out_logits

out_logits, last_state = sess.run(
    [modelnn.logits, modelnn.last_state],
    feed_dict = {
        modelnn.X: np.expand_dims(df_log.iloc[upper_b:], axis = 0),
        modelnn.hidden_layer: init_value,
    },
)
init_value = last_state
output_predict[upper_b + 1 : df_log.shape[0] + 1] = out_logits
df_log.loc[df_log.shape[0]] = out_logits[-1]
date_ori.append(date_ori[-1] + timedelta(days = 1))

In [158]:
for i in range(future_day - 1):
    out_logits, last_state = sess.run(
        [modelnn.logits, modelnn.last_state],
        feed_dict = {
            modelnn.X: np.expand_dims(df_log.iloc[-timestamp:], axis = 0),
            modelnn.hidden_layer: init_value,
        },
    )
    init_value = last_state
    output_predict[df_log.shape[0]] = out_logits[-1]
    df_log.loc[df_log.shape[0]] = out_logits[-1]
    date_ori.append(date_ori[-1] + timedelta(days = 1))

# Regarder si on peux directement faire les calculs de score, MSE sur les mesure entres 0 et 1

In [159]:
df_log = minmax.inverse_transform(output_predict)
date_ori = pd.Series(date_ori).dt.strftime(date_format = '%Y-%m-%d').tolist()

In [160]:
def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

# Attention Trading Day 

In [161]:
df_pred=pd.DataFrame(df_log)
df_pred.columns=logo_pred
df_pred.tail(3)


,AAPL_Pred,ABT_Pred,ACN_Pred,ATVI_Pred,ADBE_Pred,AMD_Pred,GOOGL_Pred,GOOG_Pred,AMZN_Pred,AAL_Pred,BLK_Pred,CBS_Pred,MSFT_Pred,FB_Pred
256,-73.524082,-3.195981,65.044697,5.827881,49.224533,2.520857,121.223985,194.057437,-663.238693,40.798665,145.304638,-0.334733,37.858610,-18.761325
257,306.589018,245.068651,394.243311,11.393528,262.143543,-6.563217,-1457.218293,-1600.532009,-175.611286,-78.161966,1103.373147,396.277595,187.043876,266.826621
258,4.996628,18.271296,444.697599,382.578691,539.763001,-6.936736,4674.088185,4270.858328,7250.370041,-383.900603,-176.860392,-202.704329,257.199079,716.336497


In [162]:
df_final=df_all.join(df_pred)
df_final.tail(3)

,Date,AAPL,ABT,ACN,ATVI,ADBE,AMD,GOOGL,GOOG,AMZN,...,ADBE_Pred,AMD_Pred,GOOGL_Pred,GOOG_Pred,AMZN_Pred,AAL_Pred,BLK_Pred,CBS_Pred,MSFT_Pred,FB_Pred
254,2016-01-06,100.70,42.56,102.16,36.79,91.02,2.505,759.33,743.62,632.65,...,94.403892,2.917860,794.352958,778.660509,661.267695,41.158033,327.898282,43.120488,57.344854,104.651667
255,2016-01-07,96.45,41.54,99.16,36.27,89.11,2.275,741.00,726.39,607.94,...,103.322109,4.633403,923.242789,910.181901,680.685575,35.568259,285.036624,26.691158,65.039900,108.637435
256,2016-01-08,96.96,40.67,98.20,35.71,87.85,2.140,730.91,714.47,607.05,...,49.224533,2.520857,121.223985,194.057437,-663.238693,40.798665,145.304638,-0.334733,37.858610,-18.761325


In [163]:
df_pred=df_final[-3:]
df_pred=df_pred.set_index('Date')
df_pred

,AAPL,ABT,ACN,ATVI,ADBE,AMD,GOOGL,GOOG,AMZN,AAL,...,ADBE_Pred,AMD_Pred,GOOGL_Pred,GOOG_Pred,AMZN_Pred,AAL_Pred,BLK_Pred,CBS_Pred,MSFT_Pred,FB_Pred
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,100.70,42.56,102.16,36.79,91.02,2.505,759.33,743.62,632.65,41.23,...,94.403892,2.917860,794.352958,778.660509,661.267695,41.158033,327.898282,43.120488,57.344854,104.651667
2016-01-07,96.45,41.54,99.16,36.27,89.11,2.275,741.00,726.39,607.94,40.45,...,103.322109,4.633403,923.242789,910.181901,680.685575,35.568259,285.036624,26.691158,65.039900,108.637435
2016-01-08,96.96,40.67,98.20,35.71,87.85,2.140,730.91,714.47,607.05,40.37,...,49.224533,2.520857,121.223985,194.057437,-663.238693,40.798665,145.304638,-0.334733,37.858610,-18.761325


In [ ]:
df_pred.to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\LSTM_All_and_Pred.csv")